In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
meta_data = pd.read_csv("Meta_Data.csv")

In [ ]:
meta_data['action'].value_counts(dropna=False)

In [ ]:
file_path = meta_data['file_path'].tolist()
action = meta_data['action'].tolist()

In [ ]:
file_path

In [ ]:
mp_hands = mp.solutions.hands # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
zero_list = [0] * 63

#for file in tqdm(file_path[:1]):
for file in tqdm(file_path):
    
    result = []
    cap = cv2.VideoCapture( file )
    
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    with mp_hands.Hands( max_num_hands = 1,
                        min_detection_confidence=0.5, 
                        min_tracking_confidence=0.5) as hands:
        frame = 0
        while True:
            success, image = cap.read()

            if success == False:
                cap.release()
                break
            
            frame = frame+1
            
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
            image.flags.writeable = False                  # Image is no longer writeable
            results = hands.process(image)                 # Make prediction
            image.flags.writeable = True                   # Image is now writeable 
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR

            if results.multi_hand_landmarks:
                
                one_frame_result = []
                
                for hand_landmarks in results.multi_hand_landmarks:
                    for ids, landmrk in enumerate(hand_landmarks.landmark):
                        #print(landmrk.x)
                        one_frame_result.append( landmrk.x )
                        one_frame_result.append( landmrk.y )
                        one_frame_result.append( landmrk.z )
                        
                result.append( one_frame_result )
                
            else:
                result.append( zero_list )
                
        a = pd.DataFrame(result)
        csv_file_name = file.replace("mp4" , "csv")
        a.to_csv(csv_file_name , index=False)

In [ ]:
csv_file_list = []

for file in file_path:
    csv_file_list.append( file.replace("mp4" , "csv") )

In [ ]:
csv_file_list

In [ ]:
meta_data['csv_file_path'] = csv_file_list

In [ ]:
meta_data

In [ ]:
meta_data.to_csv("Meta_Data_211229.csv" , index=False)

In [22]:
meta_data = pd.read_csv("Meta_Data_220105.csv")

In [23]:
file_path = meta_data['csv_file_path'].tolist()
action = meta_data['action'].tolist()

In [24]:
file_path

['./Train_Data\\1_Finger_Click\\Video_Test_10 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10 (4).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (4).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (2).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (3).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (4).csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14.csv',
 './Train_Da

In [30]:
length = []

for file in file_path:
    d = pd.read_csv(file)
    d = np.array(d)
    
    #if d.shape[0] < 40:
    #    print(file)
        
    length.append( d.shape[0] )

./Train_Data\2_Fingers_Left\Video_Test_13 (3).csv
./Train_Data\2_Fingers_Left\Video_Test_21 (3).csv
./Train_Data\2_Fingers_Right\Video_Test_10 (5).csv
./Train_Data\2_Fingers_Right\Video_Test_12 (5).csv
./Train_Data\2_Fingers_Right\Video_Test_15 (5).csv
./Train_Data\2_Fingers_Right\Video_Test_19 (3).csv
./Train_Data\2_Fingers_Right\Video_Test_20 (3).csv
./Train_Data\2_Fingers_Right\Video_Test_22.csv
./Train_Data\2_Fingers_Right\Video_Test_23.csv
./Train_Data\2_Fingers_Right\Video_Test_24.csv
./Train_Data\2_Fingers_Right\Video_Test_25.csv
./Train_Data\2_Fingers_Right\Video_Test_28.csv
./Train_Data\2_Fingers_Right\Video_Test_36.csv
./Train_Data\2_Fingers_Right\Video_Test_39.csv
./Train_Data\2_Fingers_Right\Video_Test_41.csv


In [26]:
len(length)

389

### Video Data File의 Max Frame

In [27]:
max_length = np.max(length)
max_length

115

In [28]:
min_length = np.min(length)
min_length

32

### Max Length의 길에 맞춰셔 전체 Data를 다시 만든다.   

In [62]:
def add_dummy_data(d , half, add_one_dummy_line):
    zero_data = [0] * 63 * half
    
    zero_data = np.array( zero_data )
    zero_data = zero_data.reshape(-1,63)
    
    d = np.concatenate((zero_data , d), axis=0)
    d = np.concatenate((d , zero_data), axis=0)
    
    if add_one_dummy_line:
        zero_data = [0] * 63
        zero_data = np.array( zero_data )
        zero_data = zero_data.reshape(-1,63)
        d = np.concatenate((zero_data , d), axis=0)
    
    #print(d.shape)
    return d

In [66]:
padded_file_name = []

for file in file_path:
    d = pd.read_csv(file)
    d = np.array(d)
    
    half = (max_length - d.shape[0])/2
    
    if int(half) == half:
        d = add_dummy_data(d , int(half) , False)
    else:
        d = add_dummy_data(d , int(half) , True)
        
    d = pd.DataFrame(d)
    
    file_name = file.replace(".csv" , "_padded.csv")
    padded_file_name.append(file_name)
    d.to_csv( file_name , index=False )

In [67]:
padded_file_name

['./Train_Data\\1_Finger_Click\\Video_Test_10 (2)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10 (3)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10 (4)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_10_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11 (2)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11 (3)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_11_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (2)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (3)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12 (4)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_12_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13 (2)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13 (3)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_13_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (2)_padded.csv',
 './Train_Data\\1_Finger_Click\\Video_Test_14 (3)_padded.csv',
 './Trai

In [68]:
meta_data['padded_file_name'] = padded_file_name

In [69]:
meta_data

,file_path,action,csv_file_path,padded_file_name
0,./Train_Data\1_Finger_Click\Video_Test_10 (2).mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_10 (2).csv,./Train_Data\1_Finger_Click\Video_Test_10 (2)_...
1,./Train_Data\1_Finger_Click\Video_Test_10 (3).mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_10 (3).csv,./Train_Data\1_Finger_Click\Video_Test_10 (3)_...
2,./Train_Data\1_Finger_Click\Video_Test_10 (4).mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_10 (4).csv,./Train_Data\1_Finger_Click\Video_Test_10 (4)_...
3,./Train_Data\1_Finger_Click\Video_Test_10.mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_10.csv,./Train_Data\1_Finger_Click\Video_Test_10_padd...
4,./Train_Data\1_Finger_Click\Video_Test_11 (2).mp4,1_Finger_Click,./Train_Data\1_Finger_Click\Video_Test_11 (2).csv,./Train_Data\1_Finger_Click\Video_Test_11 (2)_...
...,...,...,...,...
384,./Train_Data\Shake_Hand\Video_Test_8.mp4,Shake_Hand,./Train_Data\Shake_Hand\Video_Test_8.csv,./Train_Data\Shake_Hand\Video_Test_8_padded.csv
385,./Train_Data\Shake_Hand\Video_Test_9 (2).mp4,Shake_Hand,./Train_Data\Shake_Hand\Video_Test_9 (2).csv,./Train_Data\Shake_Hand\Video_Test_9 (2)_padde...
386,./Train_Data\Shake_Hand\Video_Test_9 (3).mp4,Shake_Hand,./Train_Data\Shake_Hand\Video_Test_9 (3).csv,./Train_Data\Shake_Hand\Video_Test_9 (3)_padde...
387,./Train_Data\Shake_Hand\Video_Test_9 (4).mp4,Shake_Hand,./Train_Data\Shake_Hand\Video_Test_9 (4).csv,./Train_Data\Shake_Hand\Video_Test_9 (4)_padde...


In [70]:
meta_data.to_csv("Meta_Data_220105.csv" , index=False)